You can just click **Runtime → Run all** to try the demo.

# Module 3 - Markov Chain: Language, Probability & Illusion

In this module, we explore how Markov chains, despite their simplicity, can generate outputs that resemble language, conversation, or even music. By modeling transitions between elements—letters, words, or notes—purely based on observed probabilities, these systems can create sequences that feel meaningful, even though no understanding is involved. This invites us to reflect on how easily structure and pattern can give the illusion of intent or intelligence.

In [ ]:
#@title system setup and data loading {display-mode: "form"}
# install packages
!pip install --quiet nltk wordcloud matplotlib ipywidgets

# import packages
import nltk
nltk.download('brown') # download the standard Brown Corpus training data set for Markov Chain

from nltk.corpus import brown
from collections import defaultdict, Counter
import random
import string

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import textwrap

from ipywidgets import Output
import asyncio
import os



import warnings
warnings.filterwarnings('ignore')

## the Brown Corpus Dataset

The Brown Corpus is a widely used text dataset compiled from American English sources in the 1960s. It includes documents from various genres—such as news, fiction, government, and academic writing—making it useful for studying general language patterns. In this demo, we use it as training data for a simple character-level Markov model. Like the MNIST data set we've seen before, it was implemented in another publicly-available Python package called nltk (natural language ToolKit)

## Section 1: Markov Chain
A Markov chain is a mathematical model where the next state depends only on the current state, not the full history. First introduced by Russian mathematician Andrey Markov in the early 1900s, it was originally used to study patterns in poetry. Today, Markov chains are widely used in fields like natural language processing, genetics, and economics. In text generation, a Markov model learns the probability of each character following a given context and uses that to generate new text, one character at a time.

In [ ]:
#@title Train a simple Markov Chain model, which probablisticly selects the next letter {display-mode: "form"}

# Prep character sequence
text = ' '.join(brown.words()).lower()
text = ''.join([c for c in text if c in string.ascii_lowercase + " .,;:!?'\n"])
text = text.replace('\n', ' ')  # unify whitespace

# Build 3-gram model: P(c_i | c_{i-3}, c_{i-2}, c_{i-1})
def train_char_markov(text, order=3):
    model = defaultdict(Counter)
    for i in range(order, len(text)):
        context = text[i - order:i]
        next_char = text[i]
        model[context][next_char] += 1
    # Normalize to probabilities
    for ctx, counter in model.items():
        total = sum(counter.values())
        model[ctx] = {c: count / total for c, count in counter.items()}
    return model

char_model = train_char_markov(text, order=3)

In [ ]:
#@title Now we can run the trained Markov model to generate a random letter string {display-mode: "form"}

def generate_trace(model, order=3, steps=30):
    seed = random.choice(list(model.keys()))
    output = seed
    trace = []

    for _ in range(steps):
        context = output[-order:]
        probs = model.get(context, {' ': 1.0})  # fallback to space
        next_char = random.choices(list(probs.keys()), weights=probs.values())[0]
        trace.append((context, probs, next_char))
        output += next_char
    return output, trace

In [ ]:
#@title Run the Markov Code {display-mode: "both"}
steps=30 # change this value if you want to generate letter series of different lengths
output_string, trace = generate_trace(char_model, steps=steps)

In [ ]:
#@title visualize the running results {display-mode: "form"}


# Define consistent character axis
full_charset = list(string.ascii_lowercase) + [' ', '.', ',', ';', ':', '!', '?', "'"]
CHAR_X = full_charset + ['[END]']

out = Output()
display(out)


cmap = cm.get_cmap('Blues')  # Darker = higher

def prob_to_color(p):
    # Manually enforce 0.0–1.0 scaling
    p_clipped = max(0.0, min(1.0, float(p)))
    rgba = cmap(p_clipped)
    return mcolors.to_hex(rgba)

def render_char_bar(i):
    context, probs, selected = trace[i]
    sentence_so_far = ''.join([t[2] for t in trace[:i+1]])

    # Prepare fixed bars
    prob_list = [probs.get(c, 0.0) for c in CHAR_X]
    x = list(range(len(CHAR_X)))

    colors = []

    for c, p in zip(CHAR_X, prob_list):
        colors.append(prob_to_color(p)) # blue shade for probability

    # === Diagnostic check for alignment ===
    assert len(prob_list) == len(CHAR_X) == len(colors), "Length mismatch"
    for idx, (c, p, col) in enumerate(zip(CHAR_X, prob_list, colors)):
        expected = prob_to_color(p)
        assert col == expected, f"Mismatch at {c}: prob={p:.3f}, color={col} ≠ {expected}"
    # === End diagnostic block ===

    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12, 4))
        ax.bar(x, prob_list, color=colors)
        ax.set_ylim(0, 1)
        ax.set_xlim(-0.5, len(CHAR_X) - 0.5)
        ax.set_xticks([])
        ax.set_ylabel('P(next char)', fontsize=12)
        ax.set_title(f"Step {i+1} — Generated so far: {sentence_so_far!r}",
                     loc='left', fontsize=13)
        fig.subplots_adjust(left=0.05, right=0.95, top=0.85, bottom=0.2)
        plt.show()

async def autoplay_trace(pause=0.2):
    for i in range(len(trace)):
        render_char_bar(i)
        await asyncio.sleep(pause)

await autoplay_trace()

## How a Markov Chain Generates Text

This animation shows how a character-level Markov chain generates text one letter at a time. The bar height and color represent the probability of each next character (letters, spaces, punctuation, etc.).

At each step, the model updates the probabilities based on the current context, then randomly selects the next character. Some word-like patterns emerge, but the full sentence often lacks meaning.

This highlights that the machine is not choosing words based on meaning, but simply following learned probabilities. Larger models can produce more human-like text, but the process remains fundamentally probabilistic.

## Section 2: Search vs Generation - "A Confused Parrot"

Nowadays, since large GenAI models such as ChatGPT, Gemni, Copilot, etc. are providing plausible answers, many people starts to use them as a search engine. Moreover, now search engine websites like google or bing are integrating AI with their search, providing summaries on the results. The difference between GenAI and Search Engine starts to blur to many users. However, from the demo above, we have prove that the generative models do not generate text based on their meanings, but based on the probability calculated from previous letter/words.

In this section, we are going to explore the difference between a search engine and a generative model, which we call **parrot\_search** and **parrot\_markov**.

Both parrots will learn from a tiny knowledge base composed of the wikipedia pages of Bach, Haydn, Mozart, and Beethoven. Then we'll see how the parrots will respond to our inputs and observe the difference between retrieving information and generating it.

### obtaining prepared files
Directly generate searchbase and training Markov chain will take a long time. Therefore, we have prepared some pre-generated files from the raw knowledge base to facilitate the Parrots to work faster. Moreover, some algorithms are complex and therefore we don't include them in this notebook, but rather prepared some pre-written codes to import.
Let's download these files.

In [ ]:
#@title Download files {display-mode: "form"}
url_header = "https://raw.githubusercontent.com/WeihaoGe1009/ml-demos-temp-inputs/main/03_markov/"


files_to_download = {
    "parrots.py": url_header + "scripts/parrots.py",
    "bwt_input_bytes.pkl": url_header + "data/bwt_index/bwt_input_bytes.pkl",
    "sa.pkl": url_header + "data/bwt_index/sa.pkl",
    "paragraph_offsets.pkl": url_header + "data/bwt_index/paragraph_offsets.pkl",
    "paragraph_map.pkl": url_header + "data/bwt_index/paragraph_map.pkl",
    "markov_model.pkl": url_header + "data/markov_model.pkl"
}

for filename, url in files_to_download.items():
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        os.system(f"wget {url}")
    else:
        print(f"{filename} already exists. Skipping.")

## Comparing the parrots:
* the **parrot\_search** looks up the original knowledge base for matching word combinations. On the other hand, the **parrot\_markov** calculated what the probability is for a word to follow another, based on the original knowledge base, and generate text probablistically.
* only **parrot\_search** will be able to say "I don't know" when it couldn't find anything in the database.
* **parrot\_markov**, on the other hand, always has something to say. it will generate words by probability without realizing any meaning, since the meaning is not something learnt in its implementation. This is so-called "illusion" in the modern GenAI, where generative AIs are "confidently" creating unreliable answers.


In [ ]:
#@title Set up Parrot_Search {display-mode: "form"}
# set up the pre-written Parrot_Search package and load the index files
# the index files are transformed from original knowledge base text,
# which are created to facilitate search
from parrots import ParrotSearchBWT, multi_keyword_search, highlight_keywords

parrot_search = ParrotSearchBWT(
    "bwt_input_bytes.pkl",
    "sa.pkl",
    "paragraph_offsets.pkl",
    "paragraph_map.pkl"
)

In [ ]:
#@title Set up Parrot_Marokv {display-mode: "form"}
from parrots import ParrotMarkovPhrase, generate_phrase_text
parrot_markov = ParrotMarkovPhrase("markov_model.pkl")

In [ ]:
#@title Compare Parrot_Search and Parrot_Markov {display-mode: "form"}


# you can change the values in keywords
keywords = ['Bach', '1750', 'German']

keywords = [kw.lower() for kw in keywords] # changing everything to lowercase

search_results = multi_keyword_search(parrot_search, keywords)
for r in search_results:
    #print("Article:", r["article"])
    wrapped_search_text = textwrap.fill(highlight_keywords(r["text"], keywords), width=80)
    print(wrapped_search_text)
    print("-" * 80)

generation_results = generate_phrase_text(parrot_markov, keywords)
wrapped_generation_text = textwrap.fill(highlight_keywords(generation_results, keywords), width=80)
print(wrapped_generation_text)

### What is the difference between our parrots?
* **Mechanism**
  * *parrot_search*: Retrieves existing sentence that contains the keywords.
  * *parrot_markov*: Generates a new paragraph based on phrase-to-phrase patterns learned from the corpus.

* **Source of Output**
  * *parrot_search: Output is taken directly from the original knowledge base (Wikipedia pages).
  * *parrot_markov*: Output is synthesized and does not exist as-is in the corpus.

* **Keyword Coverage**
  * *parrot_search*: Must contain the exact keywords.
  * *parrot_markov*: May or may not include all the keywords.

* **Output Uniqueness**
  * *parrot_search*: The same sentence can be found verbatim in the corpus.
  * *parrot_markov*: The generated text is unlikely to appear word-for-word in the original data.

* **Factuality**
  * *parrot_search*: The content is real, but may be contextually irrelevant.
  * *parrot_markov*: The output may sound plausible, but is not guaranteed to be correct or real.


## Section 3: Markov Musician - optional

Generative models not only works on texts. It can generate images, music, voice, videos, etc. Here we are going to explore a tiny simple case using existing package to generate a tiny excerpt of music.

In [ ]:
#@title install music-relevant packages (a bit slow) {display-mode: "form"}
# Install system-level FluidSynth (needed by pyfluidsynth and pretty_midi)
!apt-get install -y -qq fluidsynth > /dev/null

!pip install -q pretty_midi mido pyfluidsynth


In [ ]:
#@title Generate a music sequence based on a "prompt" of C major scale {display-mode: "form"}
import numpy as np
import pretty_midi

# Define a simple note vocabulary (C major scale)
notes = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4']
note_to_pitch = {note: pretty_midi.note_name_to_number(note) for note in notes}
pitch_to_note = {v: k for k, v in note_to_pitch.items()}

# Create a simple 1st-order Markov transition matrix with uniform probabilities
transition_matrix = np.ones((len(notes), len(notes))) / len(notes)

# Generate a note sequence
np.random.seed(42)
sequence = []
current_note = np.random.choice(notes)
sequence.append(current_note)

for _ in range(32):  # 32 notes long
    i = notes.index(current_note)
    next_note = np.random.choice(notes, p=transition_matrix[i])
    sequence.append(next_note)
    current_note = next_note


In [ ]:
#@title Convert the text sequence to music format midi {display-mode: "form"}
def create_midi(note_sequence, output_path="markov_music.mid", duration=0.5):
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    start_time = 0
    for note in note_sequence:
        pitch = note_to_pitch[note]
        note_obj = pretty_midi.Note(
            velocity=100, pitch=pitch,
            start=start_time, end=start_time + duration
        )
        instrument.notes.append(note_obj)
        start_time += duration

    pm.instruments.append(instrument)
    pm.write(output_path)
    return output_path

midi_path = create_midi(sequence)

In [ ]:
#@title Play the music {display-mode: "form"}
import pretty_midi
from IPython.display import Audio

def midi_to_audio(midi_path):
    pm = pretty_midi.PrettyMIDI(midi_path)
    audio = pm.fluidsynth()
    return Audio(audio, rate=22050)

midi_to_audio(midi_path)


## Take-home Messages
* Markov-Chain model and other generative models produce a chain of output probablistically. These models do not learn the "meanings".
* Generative model and search engine are different. Generative model do not look for matching patterns, but produces word after word probablistically
* Same priciples applies to other forms of content. (but in biology, the word (6 kmer) frequency in the S16 ribosome is a feature good enough to tell species apart and led to the discovery of Archea. how about human's work, shall these frequencies be considered as a fingerprint for people's art work, so that if the machine learns, it is violating their rights? or it just fingerprints style, which is not protected by the IP law, but still violates other rights maybe? or what if the machine learns from many different people, can we say these rights are not violated since now the frequency does not match to anyone's fingerprinting prequencies? this is counter-intuitive to Author G who does not know IP law very well, like, stealing from one person is stealing while stealing from 400 people becomes original?)

## Final Comments
In biology, the frequency of short DNA sequences—like 6-mers in the 16S ribosomal RNA—can distinguish species and even led to the discovery of Archaea.

Could word or style frequencies in human creative work function in a similar way—like a fingerprint of artistic identity?

Even if such fingerprinting works, it can be blurred by training on different input data. That leads to the question:
* If a machine learns patterns from one artist, we might say it copies.
* If it learns from 400 and the patterns become unclear, does it become “original”?
